# Module loading

In [1]:
library(IRdisplay)
library(readr)
library(fgsea)
library(dplyr)
library(reticulate)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
pd <- import("pandas")

# Settings

In [3]:
CRISPR_DATA_DIR <- Sys.getenv("PHENOPLIER_CRISPR_BASE_DIR")

In [4]:
CRISPR_DATA_DIR

[1] "/home/miltondp/projects/labs/greenelab/phenoplier/base/data/crispr_screen"

# Data loading

## Lipids gene sets

In [5]:
input_file <- Sys.getenv("PHENOPLIER_CRISPR_LIPIDS_GENE_SETS_FILE")
display(input_file)

[1] "/home/miltondp/projects/labs/greenelab/phenoplier/base/data/crispr_screen/lipid_DEG.csv"

In [6]:
all_genes_ranked <- read_csv(input_file)


── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  gene_name = col_character(),
  GFPLow_vs_UnSorted.log2FC = col_double(),
  GFPLow_vs_UnSorted.FDR = col_double(),
  GFPLow_vs_UnSorted.DEG = col_logical(),
  GFPHigh_vs_UnSorted.log2FC = col_double(),
  GFPHigh_vs_UnSorted.FDR = col_double(),
  GFPHigh_vs_UnSorted.DEG = col_logical(),
  GFPHigh_vs_GFPLow.log2FC = col_double(),
  GFPHigh_vs_GFPLow.FDR = col_double(),
  GFPHigh_vs_GFPLow.DEG = col_logical(),
  `lipid effect` = col_character(),
  rank = col_double()
)




In [7]:
orig_deg_gene_sets <- list()

for (r in unique(all_genes_ranked$rank)) {
    if (r == 0) {
        next
    }
    
    data <- all_genes_ranked[all_genes_ranked$rank == r,]
    #q <- quantile(data, 0.50, names=FALSE)
    
    orig_deg_gene_sets[[paste0("gene_set_", r)]] <- data$gene_name
}

In [8]:
length(orig_deg_gene_sets)

[1] 6

### Combine gene sets into "increase lipids" and "decrease lipids"

In [9]:
deg_gene_sets <- list()

In [10]:
# genes that increase lipids
deg_gene_sets[["gene_set_increase"]] <- c(
    orig_deg_gene_sets[["gene_set_2"]],
    orig_deg_gene_sets[["gene_set_3"]]
)

In [11]:
# genes that decrease lipids
deg_gene_sets[["gene_set_decrease"]] <- c(
    orig_deg_gene_sets[["gene_set_-2"]],
    orig_deg_gene_sets[["gene_set_-3"]]
)

In [12]:
length(deg_gene_sets)

[1] 2

In [13]:
length(deg_gene_sets[["gene_set_increase"]])

[1] 175

In [14]:
length(deg_gene_sets[["gene_set_decrease"]])

[1] 96

In [15]:
# test new increase set
new_set <- deg_gene_sets[["gene_set_increase"]]
expected_set <- union(
    orig_deg_gene_sets[["gene_set_2"]],
    orig_deg_gene_sets[["gene_set_3"]]
)

stopifnot(length(new_set) == length(unique(new_set)))

stopifnot(
    length(new_set) == 
    length(
        intersect(
            new_set,
            expected_set
        )
    )
)

In [16]:
# test new decrease set
new_set <- deg_gene_sets[["gene_set_decrease"]]
expected_set <- union(
    orig_deg_gene_sets[["gene_set_-2"]],
    orig_deg_gene_sets[["gene_set_-3"]]
)

stopifnot(length(new_set) == length(unique(new_set)))

stopifnot(
    length(new_set) == 
    length(
        intersect(
            new_set,
            expected_set
        )
    )
)

## MultiPLIER Z

In [17]:
multiplier_z = pd$read_pickle(
    Sys.getenv("PHENOPLIER_MULTIPLIER_MODEL_Z_MATRIX_FILE")
)

In [18]:
dim(multiplier_z)

[1] 6750  987

In [19]:
head(multiplier_z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GAS6,0.0000000,0,0.03943774,0,0.05047625,0.00000000,0.0000000,0.000000000,0.5909494,0.00000000,⋯,0.05012510,0.00000000,0.033407371,0.00000000,0.00000000,0.0059633917,0.34736209,0,0.00000000,0.00000000
MMP14,0.0000000,0,0.00000000,0,0.07007159,0.00000000,0.0000000,0.004904131,1.7201788,2.42359463,⋯,0.00000000,0.00000000,0.001007286,0.00000000,0.03574724,0.0000000000,0.00000000,0,0.01497801,0.00000000
DSP,0.0000000,0,0.00000000,0,0.00000000,0.04169683,0.0000000,0.005718149,0.0000000,0.00000000,⋯,0.02085321,0.00000000,0.000000000,0.00000000,0.00000000,0.0057744399,0.00000000,0,0.00000000,0.41640455
MARCKSL1,0.3052117,0,0.00000000,0,0.00000000,0.00000000,0.0000000,0.000000000,0.1618435,0.14947148,⋯,0.02713418,0.05271997,0.000000000,0.03018947,0.06088351,0.0000000000,0.00000000,0,0.00000000,0.44847996
SPARC,0.0000000,0,0.00000000,0,0.00000000,0.00000000,0.0000000,0.000000000,0.0000000,0.01401441,⋯,0.00000000,0.00000000,0.000000000,0.00000000,0.00000000,0.0000000000,0.06777859,0,0.12241734,0.06266490
CTSD,0.2778532,0,0.06459781,0,0.00000000,0.00000000,0.4694809,0.099949232,0.9717295,0.43334911,⋯,0.00000000,0.01907577,0.008904815,0.00000000,0.00000000,0.0007222884,0.00000000,0,0.00000000,0.06193245


# Prepare LVs list

In [20]:
lvs = list()
z_gene_names <- rownames(multiplier_z)

for (cidx in 1:ncol(multiplier_z)) {
    data <- multiplier_z[, cidx]
    names(data) <- z_gene_names
    # q <- quantile(data, 0.75, names=FALSE)
    q <- 0.0
    
    lvs[[paste0("LV", cidx)]] <- data[data > q]
}

# Compute enrichment on all LVs

In [21]:
n_reps = 10

In [22]:
set.seed(0)

In [23]:
results = list()

for (lv in names(lvs)) {
    repetitions = list()
    
    for (i in 1:n_reps) {
        rep_res <- fgsea(pathways = deg_gene_sets, stats = lvs[[lv]], scoreType = "pos", eps = 0.0)[order(pval), ]
        rep_res[, "leadingEdge"] <- sapply(rep_res$leadingEdge, paste, collapse=",")
        rep_res[, "lv"] <- lv
        rep_res[, "rep_idx"] <- i
        
        repetitions[[i]] <- rep_res
    }
    
    res <- do.call(rbind, repetitions)

    results[[lv]] <- res
}

In [24]:
length(results)

[1] 5

In [25]:
df <- do.call(rbind, results)

In [26]:
dim(df)

[1] 100  10

In [27]:
head(df)

pathway,pval,padj,log2err,ES,NES,size,leadingEdge,lv,rep_idx
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>
gene_set_decrease,0.1848152,0.2357642,0.09592068,0.5784179,1.126666,35,"PTBP1,KEAP1,PEX14,DLST,PCYT2,MAD2L2,GLRX5,OGDH,UBE2J2,CSK",LV1,1
gene_set_increase,0.2357642,0.2357642,0.08220549,0.5391549,1.076511,63,"CHERP,RANGAP1,HNRNPL,RPS2,E4F1,TAF1C,GATAD2A,SAFB,TAF6,FBL,LSM4,SUPT5H,CHD4,PFDN6,SLC35B2,USP39,POLR3E,POLR2C,RPLP0,SREBF2,ACACA,RPL18",LV1,1
gene_set_decrease,0.1988012,0.2517483,0.09167952,0.5784179,1.123767,35,"PTBP1,KEAP1,PEX14,DLST,PCYT2,MAD2L2,GLRX5,OGDH,UBE2J2,CSK",LV1,2
gene_set_increase,0.2517483,0.2517483,0.07871138,0.5391549,1.078565,63,"CHERP,RANGAP1,HNRNPL,RPS2,E4F1,TAF1C,GATAD2A,SAFB,TAF6,FBL,LSM4,SUPT5H,CHD4,PFDN6,SLC35B2,USP39,POLR3E,POLR2C,RPLP0,SREBF2,ACACA,RPL18",LV1,2
gene_set_decrease,0.1628372,0.2157842,0.10357633,0.5784179,1.137180,35,"PTBP1,KEAP1,PEX14,DLST,PCYT2,MAD2L2,GLRX5,OGDH,UBE2J2,CSK",LV1,3
gene_set_increase,0.2157842,0.2157842,0.08705159,0.5391549,1.087494,63,"CHERP,RANGAP1,HNRNPL,RPS2,E4F1,TAF1C,GATAD2A,SAFB,TAF6,FBL,LSM4,SUPT5H,CHD4,PFDN6,SLC35B2,USP39,POLR3E,POLR2C,RPLP0,SREBF2,ACACA,RPL18",LV1,3


## Save

In [28]:
output_file <- file.path(CRISPR_DATA_DIR, "fsgea-all_lvs.tsv")
display(output_file)

[1] "/home/miltondp/projects/labs/greenelab/phenoplier/base/data/crispr_screen/fsgea-all_lvs.tsv"

In [29]:
write_tsv(df, output_file)

# Quick analyses

## See how one LV looks like

In [30]:
df %>% filter(lv == "LV100" & pathway == "gene_set_increase") %>% arrange(desc(padj))

pathway,pval,padj,log2err,ES,NES,size,leadingEdge,lv,rep_idx
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>


## Show significant LVs

In [31]:
df_signif <- df %>% group_by(lv, pathway) %>% summarize(max_padj = max(padj)) %>% filter(max_padj < 0.05)

`summarise()` has grouped output by 'lv'. You can override using the `.groups` argument.



In [32]:
length(df_signif)

[1] 3

In [33]:
df_signif %>% arrange(max_padj)

lv,pathway,max_padj
<chr>,<chr>,<dbl>
